In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U datasets scipy ipywidgets

In [ ]:
!pip install transformers datasets torch

In [2]:
import os
import torch
from transformers import MistralTokenizer, MistralForCausalLM, Trainer, TrainingArguments
from datasets import load_dataset

device = "cuda" if torch.cuda.is_available() else "cpu"

model_name = "mistral-large-latest"
tokenizer = MistralTokenizer.from_pretrained(model_name)
model = MistralForCausalLM.from_pretrained(model_name).to(device)


dataset = load_dataset("json", data_files="finetuning_dataset.json")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

training_args = TrainingArguments(
    output_dir="./results",              
    evaluation_strategy="epoch",         
    learning_rate=2e-5,                  
    per_device_train_batch_size=4,       
    per_device_eval_batch_size=4,        
    num_train_epochs=3,                  
    weight_decay=0.01,                   
    logging_dir="./logs",                 
    save_total_limit=3,                  
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
)

trainer.train()

model.save_pretrained("./fine_tuned_mistral")
tokenizer.save_pretrained("./fine_tuned_mistral")
